In [1]:
import os
import sys
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from nltk import word_tokenize, sent_tokenize
TEXT_DATA_DIR = './writingPrompts/'

In [2]:
# Data is split between a few different files
texts = []         # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []        # list of label ids
label_text = []    # list of label texts
# Go through each directory
for fname in sorted(os.listdir(TEXT_DATA_DIR)):
    label_id = len(labels_index)
    fpath = os.path.join(TEXT_DATA_DIR, fname)
    f = open(fpath, encoding='cp850')
    t = f.read()
    i = t.find('\n\n')  # skip header in file (starts with two newlines.)
    if 0 < i:
        t = t[i:]
    texts.append(t)
    f.close()
    labels.append(label_id)
    label_text.append(fname)
print('Found %s texts.' % len(texts))

Found 6 texts.


In [3]:
label_text

['test.wp_source',
 'test.wp_target',
 'train.wp_source',
 'train.wp_target',
 'valid.wp_source',
 'valid.wp_target']

In [4]:
all_words = []
# Go through each text in turn
for ii in range(len(texts)):
    sentences = sent_tokenize(texts[ii])
    sentences = [x for x in sentences if x != ['']]
    for sent in sentences:
        words = word_tokenize(sent)
        words = [x for x in words if x != ['']]
        all_words.append(words)
len(all_words)

14331433

In [ ]:
# Phrase Detection
# Give some common terms that can be ignored in phrase detection
# For example, 'state_of_affairs' will be detected because 'of' is provided here: 
common_terms = ["of", "with", "without", "and", "or", "the", "a", "an", ".", ",", "!", "?",]
# Create the relevant phrases from the list of sentences:
phrases = Phrases(all_words, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)
# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_words])
all_sentences[5673]

In [ ]:
bigram = Phraser.load('./embedding/bigram.phr')
all_sentences = list(bigram[all_words])
all_sentences[5673]

In [5]:
model = Word2Vec(all_words, 
                 min_count=2,   # Ignore words that appear less than this
                 size=128,      # Dimensionality of word embeddings
                 workers=2,     # Number of processors (parallelisation)
                 window=5,      # Context window for words during training
                 iter=30)       # Number of epochs training over corpus

In [6]:
model.save('./embedding/w2v_128.mdl')

In [7]:
len(model.wv.vocab)

379479